In [4]:
import hydra
from omegaconf import OmegaConf
from glob import  glob
# from rosa import  predict
from rosa.data import create_io_paths, RosaDataModule
from rosa.modeling.modules import RosaLightningModule
from pytorch_lightning import Trainer


BASE_DIR = "/Users/nsofroniew/Documents/data/rosa/outputs/2023-02-13/11-35-55"
config_dir = BASE_DIR + "/.hydra"

with hydra.initialize_config_dir(config_dir=config_dir):
    config = hydra.compose(config_name="config", overrides=OmegaConf.load(config_dir + "/overrides.yaml"))

    chkpts = BASE_DIR + "/checkpoints/*.ckpt"
    chkpt = glob(chkpts)[1]

    _, output_path = create_io_paths(config.paths)

    # Create Data Module
    rdm = RosaDataModule(
        output_path,
        config=config.data_module,
    )
    rdm.setup()

    # Load model from checkpoint
    rlm = RosaLightningModule.load_from_checkpoint(
        chkpt,
        in_dim=rdm.len_input,
        out_dim=rdm.len_target,
        config=config.module,
    )
    print(rlm)

    # trainer = Trainer()
    # predictions = trainer.predict(rlm, rdm)
    # rdm.predict_dataset.predict(predictions)
    # adata = rdm.predict_dataset.adata
    # adata = predict(config, chkpt)

# display(adata)

Global seed set to 0
/Users/nsofroniew/opt/anaconda3/envs/multiomics/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/Users/nsofroniew/opt/anaconda3/envs/multiomics/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)
/var/folders/6n/b_zkz2ns3_l02s3g4lnlklxr0000gq/T/ipykernel_15318/4235644688.py:13: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume

RosaLightningModule(
  (model): RosaJointModel(
    (main): Sequential(
      (dual_embed): ParallelEmbed(
        (models): ModuleList(
          (0): Sequential(
            (layer_norm_0): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (input_embed_0): Identity()
          )
          (1): Sequential(
            (layer_norm_1): LayerNorm((3072,), eps=1e-05, elementwise_affine=True)
            (input_embed_1): InputEmbed(
              (model): Sequential(
                (projection): Linear(in_features=3072, out_features=256, bias=True)
                (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
                (dropout): Dropout(p=0.5, inplace=False)
              )
            )
          )
        )
      )
      (join_embeds): DotEmbeds()
      (feed_forward): Identity()
      (dropout): Dropout(p=0, inplace=False)
      (expression_head): ProjectionExpressionHead(
        (model): Sequential(
          (projection): Identity()
     

In [5]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from captum.attr import IntegratedGradients, InputXGradient

In [6]:
adata = rdm.predict_dataset.adata
test_obs = np.logical_not(adata.obs['train'])
test_var = np.logical_not(adata.var['train'])

In [7]:
import torch.nn as nn

class FullModel(nn.Module):
    def __init__(self, rlm):
        super(FullModel, self).__init__()
        self.model = rlm.model

    def forward(self, x1, x2):
        return self.model((x1, x2))

In [8]:
model = FullModel(rlm)
ixg = InputXGradient(model)

In [9]:
x, y = next(iter(rdm.predict_dataloader(batch_size=1)))

In [10]:
model(*x).shape

torch.Size([1, 19431])

In [11]:
y.shape

torch.Size([1, 19431])

In [12]:
x_r = tuple(x_ind.reshape(-1, x_ind.shape[-1]).requires_grad_() for x_ind in x)

In [13]:
x_r[1].shape

torch.Size([19431, 3072])

In [14]:
attribution_r = ixg.attribute(x_r)

In [15]:

attribution = tuple(a.reshape(x[0].shape[0], x[0].shape[1], -1) for a in attribution_r)

In [16]:
attribution[0].shape

torch.Size([1, 19431, 256])

In [17]:
attribution[1].shape

torch.Size([1, 19431, 3072])

In [19]:
attr = next(iter(rlm.explain_iter(rdm.predict_dataloader(batch_size=1), ixg)))

In [21]:
attr[0].shape

torch.Size([1, 19431, 256])

In [22]:
import zarr

# results_shape = tuple((len(rdm.predict_dataset), rdm.len_target, r) for r in rdm.len_input) # rdm.len_target
results_shape = tuple(rdm.predict_dataset.adata.shape + (r,) for r in rdm.len_input) # rdm.len_target
# results_shape = rdm.predict_dataset.adata.shape + (rdm.len_input,)
z = tuple(zarr.open(f'data/joint_attribution{i}.zarr', mode='w', shape=r, chunks=(1, None, None), dtype=np.float32) for i, r in enumerate(results_shape))

ind = 0
dl = rdm.predict_dataloader(batch_size=1)
for attribution in tqdm(iter(rlm.explain_iter(dl, ixg)), total=len(dl)):
    for i, a in enumerate(attribution):
        z[i][ind:ind+len(a), :, :] = a.detach().numpy() # for joint
    ind += len(a)

301it [02:57,  1.68it/s]

In [15]:
import zarr

# results_shape = tuple((len(rdm.predict_dataset), rdm.len_target, r) for r in rdm.len_input) # rdm.len_target
results_shape = tuple(rdm.predict_dataset.adata.shape + (r,) for r in rdm.len_input) # rdm.len_target
# results_shape = rdm.predict_dataset.adata.shape + (rdm.len_input,)
# z = tuple(zarr.open(f'data/joint_attribution{i}.zarr', mode='w', shape=r, chunks=(1, None, None), dtype=np.float32) for i, r in enumerate(results_shape))

ind = 0
for x, y in tqdm(iter(rdm.predict_dataloader(batch_size=1))):
    x_r = tuple(x_ind.reshape(-1, x_ind.shape[-1]).requires_grad_() for x_ind in x)
    attribution_r = ixg.attribute(x_r)
    attribution = tuple(a.reshape(x[0].shape[0], x[0].shape[1], -1) for a in attribution_r)
    for i, a in enumerate(attribution):
        pass
        # z[target, ind:ind+len(x), :] = attribution # for gene dataset
        # z[i][ind:ind+len(a), :, :] = a.detach().numpy() # for cell dataset
        #### for joint have to do something more clever ...... maybe swap iterators
    ind += len(a)

  4%|▍         | 16/369 [00:08<02:59,  1.97it/s]

In [ ]:
import zarr

results_shape = (len(rdm.predict_dataset), 50, rdm.len_input) # rdm.len_target
# results_shape = rdm.predict_dataset.adata.shape + (rdm.len_input,)
z = zarr.zeros(results_shape, chunks=(1, None, None), dtype=np.float32)

ind = 0
for x, y in tqdm(iter(rdm.predict_dataloader(batch_size=1)), leave=False):
    for target in tqdm(range(50)):
        x.requires_grad_()
        attribution = ixg.attribute(x, target=target)
        attribution = attribution.detach().numpy()
        # z[target, ind:ind+len(x), :] = attribution # for gene dataset
        z[ind:ind+len(x), target, :] = attribution # for cell dataset
        #### for joint have to do something more clever ...... maybe swap iterators
    ind += len(x)

In [30]:
attr = next(iter(explain_iter))

  0%|          | 0/369 [00:00<?, ?it/s]

In [33]:
attr[1].shape

torch.Size([1, 19431, 3072])

In [17]:
for attr in iter(explain_iter):
    pass

  9%|▊         | 32/369 [00:16<02:57,  1.90it/s]

In [ ]:
explain_iter

In [ ]:
import zarr

# results_shape = tuple((len(rdm.predict_dataset), rdm.len_target, r) for r in rdm.len_input) # rdm.len_target
results_shape = tuple(rdm.predict_dataset.adata.shape + (r,) for r in rdm.len_input) # rdm.len_target

z = tuple(zarr.open(f'data/joint_attribution{i}.zarr', mode='r', shape=r, chunks=(1, None, None), dtype=np.float32) for i, r in enumerate(results_shape))


In [ ]:
import dask.array as da

# results_shape = tuple((len(rdm.predict_dataset), rdm.len_target, r) for r in rdm.len_input) # rdm.len_target
results_shape = tuple(rdm.predict_dataset.adata.shape + (r,) for r in rdm.len_input) # rdm.len_target

z = tuple(da.from_zarr(f'data/joint_attribution{i}.zarr') for i, r in enumerate(results_shape))


In [ ]:
np.mean(np.abs(z[1]), axis=(0, 1))

In [ ]:
z_avg = tuple(np.asarray(da.abs(z_i).mean(axis=(0, 1))) for z_i in z)

In [ ]:
z[1].shape

In [ ]:
plt.plot(np.arange(len(z_avg[0])), z_avg[0]);
plt.xlabel('n PC');
plt.ylabel('mean attribution score');

In [ ]:
plt.plot(np.arange(len(z_avg[1])), np.sort(z_avg[1])[::-1]);
plt.xlabel('embed dim');
plt.ylabel('mean attribution score');

In [ ]:
indices = np.argsort(z_avg[1])
z = (z[0], z[1][:, :, indices])

In [ ]:
z = tuple(np.asarray(z_ind[:, :, :50]) for z_ind in z)

In [ ]:
C = np.zeros((z[0].shape[2], z[1].shape[2]))

for t in tqdm(range(z[0].shape[2])):
    for s in tqdm(range(z[1].shape[2])):
        C[t, s] = np.corrcoef(z[0][test_obs][:, test_var, t].flatten(), z[1][test_obs][:, test_var, s].flatten())[0, 1]

In [ ]:
import seaborn as sns

In [ ]:
plt.imshow(C)

In [ ]:
sns.clustermap(C)

## Full model

In [ ]:
from sklearn.decomposition import PCA


train_cells = adata.obs['train']
train_genes = adata.var['train']
adata_split = adata[train_cells, train_genes]

# fit pca on training data
pca = PCA()
pca.fit(adata_split.X)

# compute scores for all cells
pca_expression = pca.transform(adata[:, train_genes].X)

# # add cell embeddings to obsm
# n_pcs = config.pcs
# n_pcs = min(n_pcs, pca_expression.shape[1] - 1)
# adata.obsm["embedding"] = pca_expression[:, :n_pcs]
# adata.uns["obs_embedding_pca"] = {
#     "explained_variance": np.cumsum(pca.explained_variance_ratio_)[n_pcs]
# }

In [ ]:
from enformer_pytorch import Enformer


MODEL_PT = "EleutherAI/enformer-official-rough"
SEQ_EMBED_DIM = 896
EMBED_DIM = 3072
TSS = int(SEQ_EMBED_DIM // 2)

In [ ]:
import torch.nn as nn
import torch

class FullModel(nn.Module):
    def __init__(self, rlm, pca):
        super(FullModel, self).__init__()
        self.input_mean = torch.from_numpy(pca.mean_)
        self.input_weights = torch.from_numpy(pca.components_[:256])
        self.enformer = Enformer.from_pretrained(MODEL_PT, output_heads=dict(), use_checkpointing=False)
        self.tss = TSS
        self.model = rlm.model

    def forward(self, x1, x2):
        x1 = x1 - self.input_mean
        x1 = torch.einsum('ij, kj -> ik', x1, self.input_weights)
        _, x2 = self.enformer(x2, return_embeddings=True)
        x2 = x2[:, self.tss]
        return self.model((x1, x2))

In [ ]:
model = FullModel(rlm, pca)

In [ ]:
from rosa.utils import score_predictions, plot_expression_and_correlation, plot_marker_gene_heatmap


adata_test, results = score_predictions(adata)

In [ ]:
plot_expression_and_correlation(adata_test, results)

In [ ]:
import numpy as np


marker_genes = adata_test.var[adata_test.var['highly_variable']]['feature_name'].values
np.random.seed(42)
marker_genes = np.random.choice(marker_genes, 50)

plot_marker_gene_heatmap(adata_test, marker_genes)

In [ ]:
marker_genes_dict = adata_test.obs.set_index('label').to_dict()['marker_feature_name']
plot_marker_gene_heatmap(adata_test, marker_genes_dict)